In [ ]:
import os
import papermill as pm
from logger import logger

## Generate Embeddings

In [ ]:
notebook = "embeddings.ipynb"
pm.inspect_notebook(notebook)

In [ ]:
# model_name = "all-MiniLM-L6-v2"
# model_name = "all-mpnet-base-v2"
model_name = "BAAI/bge-large-en-v1.5"


for strategy in ["mean"]:
    logger.info(
        f"Running {notebook} for {model_name} model with {strategy} pooling strategy"
    )
    pm.execute_notebook(
        input_path="embeddings.ipynb",
        output_path=os.path.join(
            "..", "artifacts", "notebooks", f'{model_name.replace("/","_")}_{strategy}_emb.ipynb'
        ),
        parameters={"MODEL_NAME": model_name, "POOLING_STRATEGY": strategy},
    )

2024-06-12 15:04:50,388 - Running embeddings.ipynb for BAAI/bge-large-en-v1.5 model with mean pooling strategy
Black encountered an error, skipping formatting (module 'black' has no attribute 'FileMode')
Executing:  28%|██▊       | 5/18 [00:04<00:11,  1.13cell/s]


PapermillExecutionError: 
---------------------------------------------------------------------------
Exception encountered at "In [4]":
  Cell In[4], line 26
    f"healthhub_{MODEL_NAME.replace("/","_")}_{POOLING_STRATEGY}_embeddings_small_clean.parquet",
                                     ^
SyntaxError: f-string: unmatched '('



## Evaluate Similarities

In [ ]:
notebook = "similarity.ipynb"
pm.inspect_notebook(notebook)

In [ ]:
for strategy in ["mean", "max"]:
    logger.info(
        f"Running {notebook} for {model_name} model with {strategy} pooling strategy"
    )
    pm.execute_notebook(
        input_path="similarity.ipynb",
        output_path=os.path.join(
            "..", "artifacts", "notebooks", f"{model_name}_{strategy}_sim.ipynb"
        ),
        parameters={"MODEL_NAME": model_name, "POOLING_STRATEGY": strategy},
    )